#   Tool Calling (Function Calling) + 에이전트(Agent) 개념

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---

## **Tool Calling**

- **Tool Calling**은 LLM이 외부 시스템과 상호작용하기 위한 **함수 호출 메커니즘**

- LLM은 정의된 도구나 함수를 통해 **외부 시스템과 통신**하고 작업을 수행

- **Tool calling**은 모델이 시스템과 직접 상호작용할 수 있게 하는 기능

- **구조화된 출력**을 통해 API나 데이터베이스와 같은 시스템 요구사항 충족

- **스키마 기반 응답**으로 시스템 간 효율적 통신 가능


![Tool Calling Concept](https://python.langchain.com/assets/images/tool_calling_concept-552a73031228ff9144c7d59f26dedbbf.png)


[참조] https://python.langchain.com/docs/concepts/tool_calling/

---

### 1. **Tool Creation** (`@tool` 데코레이터 사용)

- **@tool 데코레이터**로 함수에 스키마 정보 추가

- **함수와 스키마** 간 자동 연결로 도구 생성

`(1) 간단한 날씨 예제`

In [4]:
from langchain_core.tools import tool
from typing import Literal

@tool
def get_weather(city: Literal["서울", "부산", "대구", "인천", "광주"]):
    """한국 주요 도시의 날씨 정보를 가져옵니다."""
    weather_data = {
        "서울": "맑음",
        "부산": "흐림",
        "대구": "맑음",
        "인천": "비",
        "광주": "구름많음"
    }
    
    if city in weather_data:
        return f"{city} 날씨는 {weather_data[city]}"
    else:
        raise AssertionError("지원하지 않는 도시입니다")

In [5]:
type(get_weather)

langchain_core.tools.structured.StructuredTool

In [7]:
# 도구 실행
get_weather.invoke("대전")

ValidationError: 1 validation error for get_weather
city
  Input should be '서울', '부산', '대구', '인천' or '광주' [type=literal_error, input_value='대전', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/literal_error

`(2) DB 검색 예제`

In [8]:
# 벡터 저장소 로드 
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

chroma_db = Chroma(
    collection_name="db_korean_cosine_metadata",
    embedding_function=embeddings,
    persist_directory="./chroma_db",
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [9]:
# 검색기 지정하여 테스트 
chroma_k_retriever = chroma_db.as_retriever(
    search_kwargs={"k": 2},
)

query = "리비안은 언제 사업을 시작했나요?"
retrieved_docs = chroma_k_retriever.invoke(query)

print(f"쿼리: {query}")
print("검색 결과:")
for doc in retrieved_docs:
    print(f"- {doc.page_content} [출처: {doc.metadata['source']}]")

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


쿼리: 리비안은 언제 사업을 시작했나요?
검색 결과:
- <Document>
- **회사 유형:** 상장
- **거래소:** NASDAQ: RIVN
- **설립:** 2009년 6월, 플로리다 주 록ledge
- **설립자:** R. J. 스캐린지
- **본사:** 미국 캘리포니아 주 어바인
- **서비스 지역:** 북미
- **주요 인물:** R. J. 스캐린지 (CEO)
- **제품:** 전기 자동차, 배터리
- **생산량 (2023):** 57,232대
- **서비스:** 전기 자동차 충전, 자동차 보험
- **수익 (2023):** 44억 3천만 미국 달러
- **순이익 (2023):** -54억 미국 달러
- **총 자산 (2023):** 168억 미국 달러
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]
- <Document>
Rivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.

**주요 정보:**
</Document>
<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source> [출처: data\리비안_KR.md]


In [10]:
# DB 검색하는 사용자 정의 도구 생성
from langchain_core.tools import tool

@tool
def search_db(query: str):
    """리비안, 테슬라 회사에 대한 정보를 관련 데이터베이스에서 검색합니다."""
    return chroma_k_retriever.invoke(query)

# 도구 실행
search_db.invoke("리비안은 언제 사업을 시작했나요?")

[Document(id='e8cc506b-7b09-436c-892f-f5fb6c3a64bb', metadata={'source': 'data\\리비안_KR.md', 'company': '리비안', 'language': 'ko'}, page_content="<Document>\n- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- **설립:** 2009년 6월, 플로리다 주 록ledge\n- **설립자:** R. J. 스캐린지\n- **본사:** 미국 캘리포니아 주 어바인\n- **서비스 지역:** 북미\n- **주요 인물:** R. J. 스캐린지 (CEO)\n- **제품:** 전기 자동차, 배터리\n- **생산량 (2023):** 57,232대\n- **서비스:** 전기 자동차 충전, 자동차 보험\n- **수익 (2023):** 44억 3천만 미국 달러\n- **순이익 (2023):** -54억 미국 달러\n- **총 자산 (2023):** 168억 미국 달러\n</Document>\n<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source>"),
 Document(id='c2c7dbe9-b8a6-4dca-9ae5-e284009b89b2', metadata={'company': '리비안', 'language': 'ko', 'source': 'data\\리비안_KR.md'}, page_content="<Document>\nRivian Automotive, Inc.는 2009년에 설립된 미국의 전기 자동차 제조업체, 자동차 기술 및 야외 레크리에이션 회사입니다.\n\n**주요 정보:**\n</Document>\n<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source>")]

---

### 2. **Tool Binding** (모델에 Tool 연결)

- **모델-도구 연결**로 입력 스키마 자동 인식

- **스키마 기반 검증**으로 올바른 입력 보장

In [28]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama import ChatOllama

# model = ChatGoogleGenerativeAI(
#     model="gemini-2.0-flash",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
#     # other params...
# )

model = ChatOllama(model="qwen3:0.6b")

# 모델
# model = ChatOpenAI(model="gpt-4.1-mini",temperature=0)

# 도구 목록
tools = [get_weather, search_db]

# 도구를 모델에 바인딩 (bind_tools 메소드 사용)
model_with_tools = model.bind_tools(tools)

# 사용자 쿼리를 모델에 전달하여 도구를 호출
result = model_with_tools.invoke("서울 날씨 어때?")

print(result)

content="<think>\nOkay, the user is asking for the weather in Seoul. Let me check the available tools. There's a function called get_weather that takes a city parameter from the list of major cities. The cities are Seoul, Busan, Daejeon, Jeju, and Gangwon. Since the user specified Seoul, I should use that function with the city set to Seoul. No need to use the search_db function here. I'll format the tool call accordingly.\n</think>\n\n" additional_kwargs={} response_metadata={'model': 'qwen3:0.6b', 'created_at': '2025-07-02T01:09:33.7100198Z', 'done': True, 'done_reason': 'stop', 'total_duration': 9789990800, 'load_duration': 5392798300, 'prompt_eval_count': 235, 'prompt_eval_duration': 778298900, 'eval_count': 119, 'eval_duration': 3614919800, 'model_name': 'qwen3:0.6b'} id='run--f23fdff3-46fc-4802-a02c-8f4c98c2cdd0-0' tool_calls=[{'name': 'get_weather', 'args': {'city': '서울'}, 'id': 'e161e60b-d174-430e-a693-b18c824df3bf', 'type': 'tool_call'}] usage_metadata={'input_tokens': 235, 'o

In [29]:
result.tool_calls

[{'name': 'get_weather',
  'args': {'city': '서울'},
  'id': 'e161e60b-d174-430e-a693-b18c824df3bf',
  'type': 'tool_call'}]

In [ ]:
model_with_tools.invoke("안녕")

AIMessage(content='<think>\nOkay, the user said "안녕" which translates to "Hello" in Korean. I need to respond appropriately. Since they\'re greeting me, I should acknowledge their greeting. Let me check if there\'s any specific function they need, but looking at the tools provided, there\'s get_weather and search_db. Neither of these functions are related to a greeting. So I should just respond politely. No tool calls needed here.\n</think>\n\n안녕하세요! 어떤 문제나 도움을 요청하시면 언제든지 말씀해주세요. 😊', additional_kwargs={}, response_metadata={'model': 'qwen3:0.6b', 'created_at': '2025-07-02T01:10:05.0872739Z', 'done': True, 'done_reason': 'stop', 'total_duration': 8740248000, 'load_duration': 30448100, 'prompt_eval_count': 229, 'prompt_eval_duration': 45395200, 'eval_count': 112, 'eval_duration': 8663959400, 'model_name': 'qwen3:0.6b'}, id='run--d90b1c6e-4309-46f6-837b-45b18e935791-0', usage_metadata={'input_tokens': 229, 'output_tokens': 112, 'total_tokens': 341})

---

### 3. **Tool Calling** (모델이 Tool을 사용하는 경우)

- **스키마 기반 응답** 생성으로 정확한 입력 형식 준수

- **자동 유효성 검증**으로 오류 방지

- **구조화된 출력** 생성으로 시스템 호환성 보장

In [16]:
# 결과 출력
for k in dict(result).keys():
    print(f"{k}: ")
    print(dict(result)[k])
    print("-"*100)

content: 

----------------------------------------------------------------------------------------------------
additional_kwargs: 
{'tool_calls': [{'id': 'call_A9zwGeae9GDTMRRiVjgXeoi5', 'function': {'arguments': '{"city":"서울"}', 'name': 'get_weather'}, 'type': 'function'}], 'refusal': None}
----------------------------------------------------------------------------------------------------
response_metadata: 
{'token_usage': {'completion_tokens': 14, 'prompt_tokens': 109, 'total_tokens': 123, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-Bofu4ebKbRL3XMkkRvPiIr5wdc1Wd', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}
------------------------------------------------------------------------------------------

In [17]:
# tool_calls 출력
pprint(result.tool_calls)

[{'args': {'city': '서울'},
  'id': 'call_A9zwGeae9GDTMRRiVjgXeoi5',
  'name': 'get_weather',
  'type': 'tool_call'}]


In [18]:
# DB 검색 도구 호출
search_result = model_with_tools.invoke("리비안은 언제 사업을 시작했나요?")

# 검색 결과 출력
print("검색 결과:")
for k in dict(search_result).keys():
    print(f"{k}: ")
    print(dict(search_result)[k])
    print("-"*100)

# tool_calls 출력
pprint(search_result.tool_calls)

검색 결과:
content: 

----------------------------------------------------------------------------------------------------
additional_kwargs: 
{'tool_calls': [{'id': 'call_E3ekLKmMfrh7cysvWoqIYGG5', 'function': {'arguments': '{"query":"리비안 사업 시작"}', 'name': 'search_db'}, 'type': 'function'}], 'refusal': None}
----------------------------------------------------------------------------------------------------
response_metadata: 
{'token_usage': {'completion_tokens': 18, 'prompt_tokens': 114, 'total_tokens': 132, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BofwXgzGoBM2LyxLLc0KKlSOmnwXS', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}
-----------------------------------------------------------------------------

---

### 4. **Tool Execution**  (Tool이 호출된 경우 실행)

- **인자 기반 실행**으로 도구 기능 수행

- **모델 제공 파라미터**로 자동화된 실행

- **실행 결과** 처리 및 반환

`(1) 함수의 인자를 직접 전달`

In [19]:
# 함수의 인자를 직접 전달하는 방식으로 실행 -> 도구를 직접 호출
result.tool_calls[0]['args']

{'city': '서울'}

In [20]:
get_weather.invoke(result.tool_calls[0]['args'])

'서울 날씨는 맑음'

In [ ]:
search_result.tool_calls[0]['args']

{'query': '리비안 사업 시작'}

In [22]:
search_db.invoke(search_result.tool_calls[0]['args'])

[Document(id='e8cc506b-7b09-436c-892f-f5fb6c3a64bb', metadata={'source': 'data\\리비안_KR.md', 'language': 'ko', 'company': '리비안'}, page_content="<Document>\n- **회사 유형:** 상장\n- **거래소:** NASDAQ: RIVN\n- **설립:** 2009년 6월, 플로리다 주 록ledge\n- **설립자:** R. J. 스캐린지\n- **본사:** 미국 캘리포니아 주 어바인\n- **서비스 지역:** 북미\n- **주요 인물:** R. J. 스캐린지 (CEO)\n- **제품:** 전기 자동차, 배터리\n- **생산량 (2023):** 57,232대\n- **서비스:** 전기 자동차 충전, 자동차 보험\n- **수익 (2023):** 44억 3천만 미국 달러\n- **순이익 (2023):** -54억 미국 달러\n- **총 자산 (2023):** 168억 미국 달러\n</Document>\n<Source>이 문서는 미국 전기차 회사인 '리비안'에 대한 문서입니다.</Source>"),
 Document(id='4ba9002a-df79-4498-b120-07b63fc04dc9', metadata={'company': '리비안', 'source': 'data\\리비안_KR.md', 'language': 'ko'}, page_content="<Document>\n**역사**\n\n**초창기 (2009–15):**\n\n- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립.\n- 2011년 Rivian Automotive로 사명 변경.\n- 처음에는 스포츠카 프로토타입(R1)에 집중했지만 전기 및 자율 주행 차량으로 전환.\n\n**생산 준비 (2016–20):**\n\n- 2017년 일리노이 주 노멀에 있는 이전 Mitsubishi Motors 제조 공장을 1,600만 달러에 인수.\n- 2017년 12월, 첫 두 제품인 R

`(2) ToolCall 객체를 전달 전달`

In [23]:
result.tool_calls[0]

{'name': 'get_weather',
 'args': {'city': '서울'},
 'id': 'call_A9zwGeae9GDTMRRiVjgXeoi5',
 'type': 'tool_call'}

In [24]:
# ToolCall 객체를 전달 전달하는 방식으로 실행 -> ToolMessage 객체를 반환
get_weather.invoke(result.tool_calls[0])

ToolMessage(content='서울 날씨는 맑음', name='get_weather', tool_call_id='call_A9zwGeae9GDTMRRiVjgXeoi5')

In [25]:
search_db.invoke(search_result.tool_calls[0])

ToolMessage(content='[Document(id=\'e8cc506b-7b09-436c-892f-f5fb6c3a64bb\', metadata={\'language\': \'ko\', \'source\': \'data\\\\리비안_KR.md\', \'company\': \'리비안\'}, page_content="<Document>\\n- **회사 유형:** 상장\\n- **거래소:** NASDAQ: RIVN\\n- **설립:** 2009년 6월, 플로리다 주 록ledge\\n- **설립자:** R. J. 스캐린지\\n- **본사:** 미국 캘리포니아 주 어바인\\n- **서비스 지역:** 북미\\n- **주요 인물:** R. J. 스캐린지 (CEO)\\n- **제품:** 전기 자동차, 배터리\\n- **생산량 (2023):** 57,232대\\n- **서비스:** 전기 자동차 충전, 자동차 보험\\n- **수익 (2023):** 44억 3천만 미국 달러\\n- **순이익 (2023):** -54억 미국 달러\\n- **총 자산 (2023):** 168억 미국 달러\\n</Document>\\n<Source>이 문서는 미국 전기차 회사인 \'리비안\'에 대한 문서입니다.</Source>"), Document(id=\'4ba9002a-df79-4498-b120-07b63fc04dc9\', metadata={\'source\': \'data\\\\리비안_KR.md\', \'language\': \'ko\', \'company\': \'리비안\'}, page_content="<Document>\\n**역사**\\n\\n**초창기 (2009–15):**\\n\\n- 2009년 R. J. 스캐린지가 Mainstream Motors로 설립.\\n- 2011년 Rivian Automotive로 사명 변경.\\n- 처음에는 스포츠카 프로토타입(R1)에 집중했지만 전기 및 자율 주행 차량으로 전환.\\n\\n**생산 준비 (2016–20):**\\n\\n- 2017년 

---

###  Tool Calling 사용 시 **고려사항**

- **모델 호환성**이 Tool Calling 성능에 직접 영향

- **명확한 도구 정의**가 모델의 이해도와 활용도 향상

- **단순한 기능**의 도구가 더 효과적으로 작동

- **과다한 도구**는 모델 성능 저하 유발

# ~ 10:20분까지

---

## **Agent**

- **LLM(대규모 언어 모델)** 을 의사결정 엔진으로 사용하여 작업을 수행하는 시스템

- 모델은 입력된 데이터를 분석하여 **맥락에 맞는 의사결정**을 수행

- 시스템은 사용자의 요청을 이해하고 **적절한 해결책**을 제시

- 복잡한 작업을 자동화하여 **업무 효율성**을 높일 수 있음 

---

### 1. **AgentExecutor** 

- **AgentExecutor**는 LangChain의 기본 에이전트 실행 시스템

- 에이전트의 **계획-실행-관찰** 사이클을 자동으로 관리

- 에이전트의 행동을 **모니터링**하고 결과를 반환

`(1) 추가 도구 정의`

- **@tool 데코레이터**를 사용해 계산(파이썬 코드 실행) 기능을 가진 **커스텀 도구를 정의**

- 데코레이터를 통해 함수가 **Tool Calling 시스템에 등록**되어 LLM이 호출 가능

In [31]:
@tool
def calculate(expression: str) -> float:
    """수학 계산을 수행합니다."""
    return eval(expression)

In [32]:
# 도구 실행 
calculate.invoke("3+2")

5

`(2) 프롬프트 템플릿`

- **프롬프트 템플릿**은 에이전트의 **기본 행동과 응답 방식**을 정의하는 지침

- 에이전트의 **일관된 응답**과 **효율적인 도구 사용**을 위한 기본 틀 제공

In [33]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 사용자의 요청을 처리하는 AI Assistant입니다."),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

# 프롬프트 템플릿 실행
response = prompt.invoke({
    "input": "부산 날씨 어때?",
    "agent_scratchpad": []  # 에이전트 스크래치패드 (메시지 기록)
})

# 프롬프트 템플릿 실행 결과
pprint(response.messages)

[SystemMessage(content='당신은 사용자의 요청을 처리하는 AI Assistant입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산 날씨 어때?', additional_kwargs={}, response_metadata={})]


`(2) LLM 지정`

- **ChatGPT 모델**이 에이전트의 **핵심 추론 엔진**으로 사용됨

- 모델은 사용자 입력을 분석하고 **적절한 도구를 선택**하여 작업 수행

In [34]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4.1-mini",temperature=0)

`(3) Agent (에이전트) 생성`

- **에이전트**는 LLM과 도구를 **통합**하여 복잡한 작업을 수행하는 시스템

- 프롬프트 템플릿을 기반으로 **사용자 요청을 해석**하고 적절한 도구 선택

- 도구 실행 결과를 분석하여 **최종 응답을 생성**하는 워크플로우 구현

In [35]:
from langchain.agents import create_tool_calling_agent

# 도구 목록 생성 
tools = [get_weather, search_db, calculate]

# 에이전트 생성 (도구 호출)
agent = create_tool_calling_agent(llm, tools, prompt)

`(4) AgentExecutor (에이전트 실행기) 생성`

- **AgentExecutor**는 에이전트의 **작업 흐름을 관리**하고 결과를 처리하는 컴포넌트

- 사용자 입력부터 최종 출력까지의 **전체 프로세스를 조율**하고 제어

- 에러 처리, 로깅, 결과 포맷팅 등 **시스템 운영에 필요한 기능** 제공

In [36]:
from langchain.agents import AgentExecutor

# 에이전트 실행기 생성
agent_executor = AgentExecutor(
    agent=agent,      # 도구 호출 에이전트
    tools=tools,      # 도구 목록
    verbose=True,     # 상세 로그 출력
    )

In [37]:
# 에이전트 실행
response = agent_executor.invoke(
    {"input": "서울의 날씨는 어떤가요?"}
)



> Entering new AgentExecutor chain...

Invoking: `get_weather` with `{'city': '서울'}`


서울 날씨는 맑음서울의 날씨는 맑음입니다. 더 궁금한 점 있으신가요?

> Finished chain.


In [38]:
# 에이전트 실행 결과 출력
pprint(response)

{'input': '서울의 날씨는 어떤가요?', 'output': '서울의 날씨는 맑음입니다. 더 궁금한 점 있으신가요?'}


In [39]:
# 에이전트 실행기 생성
agent_executor = AgentExecutor(
    agent=agent,      # 도구 호출 에이전트
    tools=tools,      # 도구 목록
    verbose=True,
    return_intermediate_steps=True  # 중간 단계 반환 (기본값 False)
    )

response = agent_executor.invoke(
    {"input": "32 더하기 18은 얼마인가요?"}
)



> Entering new AgentExecutor chain...

Invoking: `calculate` with `{'expression': '32+18'}`


5032 더하기 18은 50입니다.

> Finished chain.


In [40]:
# 에이전트 실행 결과 출력
pprint(response)

{'input': '32 더하기 18은 얼마인가요?',
 'intermediate_steps': [(ToolAgentAction(tool='calculate', tool_input={'expression': '32+18'}, log="\nInvoking: `calculate` with `{'expression': '32+18'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_jHpne1B1DhKBZMPcIuUzSfxZ', 'function': {'arguments': '{"expression":"32+18"}', 'name': 'calculate'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'service_tier': 'default'}, id='run--cdb5de1e-c610-4d3b-888c-c25ee3ad9803', tool_calls=[{'name': 'calculate', 'args': {'expression': '32+18'}, 'id': 'call_jHpne1B1DhKBZMPcIuUzSfxZ', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'calculate', 'args': '{"expression":"32+18"}', 'id': 'call_jHpne1B1DhKBZMPcIuUzSfxZ', 'index': 0, 'type': 'tool_call_chunk'}])], tool_call_id='call_jHpne1B1DhKBZMPcIuUzSfxZ'),
                         50)],
 'output': '32

---

### 2. **LangGraph** 

- **LangGraph**는 LangChain의 확장 도구로 **고급 에이전트 개발**을 지원

- **그래프 기반 워크플로우**를 통해 복잡한 에이전트 로직을 구현할 수 있음 

- 상태 관리와 **타입 안전성**을 통해 안정적인 에이전트 실행을 보장

- AgentExecutor보다 더 **유연한 사용자 정의**가 가능함 

`(1) 도구 실행 에이전트 정의`

- LangGraph의 **react agent executor**는 메시지 목록으로 상태를 관리

- 에이전트의 출력에 **도구 호출**이 없을 때까지 메시지를 계속 처리함

- 시작 시 **초기 메시지 목록**을 입력으로 사용

- 실행 결과로 전체 **대화 기록**을 포함한 그래프 상태를 반환

- **메시지 기반 상태** 관리를 통해 에이전트의 실행 흐름을 체계적으로 제어

In [45]:
from langgraph.prebuilt import create_react_agent

# 도구 목록 생성 
tools = [get_weather, calculate, search_db]

# 도구 실행 에인전트 생성
langgraph_agent_executor = create_react_agent(llm, tools)

`(2) 에이전트 실행`

In [46]:
# 에이전트 실행
messages = langgraph_agent_executor.invoke(
    {"messages": [("human", "32 곱하기 18은 얼마인가요?")]}
)

# 에이전트 실행 결과 출력
pprint(messages)

{'messages': [HumanMessage(content='32 곱하기 18은 얼마인가요?', additional_kwargs={}, response_metadata={}, id='a450c767-857b-4600-be19-88abd9304a2c'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_b3VU85gMrkCgCf2wFslX8nUr', 'function': {'arguments': '{"expression":"32 * 18"}', 'name': 'calculate'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 134, 'total_tokens': 150, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_658b958c37', 'id': 'chatcmpl-Bogd5uFjQRLA2FmT7ivq8uDJ73bL0', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a4a85af4-00ea-40ab-871f-933aa54b928b-0', tool_calls=[{'name': 'calculate', 'args': {'expression': '32 * 18'

In [47]:
print(messages['messages'][-1].content)

32 곱하기 18은 576입니다.
